<p style="text-align: left; font-size: 32px; font-weight: bold;">
    Creating animated gifs with NEXRAD Level 2 radar data using Py-ART
</p>

# Overview
### Within this notebook, we will cover:
#### 1: Accessing NEXRAD data from AWS
#### 2: How to read the data into Py-ART and create plots
#### 3: How to create animated gifs with acquired radar data from Py-ART

# Prerequisites 


| Concepts | Importance | Notes |
| -------- | ---------- | ----- |
| [Quickstart: Zero to Python](https://foundations.projectpythia.org/foundations/quickstart.html) | Required | For loops, lists |
| [Matplotlib Basics](https://link-to-matplotlib-basics) | Required | Basic plotting |
| [Py-ART Basics](https://link-to-pyart-basics) | Required | IO/Visualization |

* **Time to learn:** 30 Minutes


# Imports for animated gif making in PyArt 

In [77]:
import pyart
import fsspec
import matplotlib.pyplot as plt
import imageio
import os
from io import BytesIO
import warnings
import cartopy.crs as ccrs
from metpy.plots import USCOUNTIES
from scipy.interpolate import griddata
import numpy as np
warnings.filterwarnings("ignore")

----

### Set up the AWS S3 filysystem
This allows you to pull nexrad-level-2 data files from the AWS repository.

In [81]:
fs = fsspec.filesystem("s3", anon=True)

### Selecting your date, station, and time period

Once the file system is set up, you can use the following code to specify what time, date, and station you'd like to retrieve data for

For this example, we will use data from NWS Chicago (KLOT) from 04 UTC, June 5th, 2024

The frames variable is an empty list that is created so we can append (add to) the frames to the list during the gif creation process

In [92]:
date = "2024/06/05" #YYYY/MM/DD
station = "KLOT"

# Specify the start and end hours
start_hour = 4
end_hour = 10

# Generate the list of files for the specified range of hours
files = []
for hour in range(start_hour, end_hour + 1):
    hour_str = f"{hour:02d}"
    files += sorted(fs.glob(f"s3://noaa-nexrad-level2/{date}/{station}/{station}{date.replace('/', '')}_{hour_str}*"))
files



['noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_040442_V06',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_040926_V06',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_041403_V06',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_041840_V06',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_042316_V06',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_042800_V06',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_043244_V06',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_043728_V06',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_044158_V06',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_044635_V06',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_045106_V06',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_045532_V06',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_045958_V06',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_045958_V06_MDM',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_050423_V06',
 'noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_0

In [94]:
frames = []

### OPTIONAL* setting up individual locations to be plotted on your map

In this example, two plots are pointed due to their relevance to the example weather event: The ATMOS facility at Argonne National Laboratory, Lemont, IL, and Sawmill Creek in Darien, IL. The latitude and longitude for each site should be placed at the same index (IE. Index 0 in both latitude and longitude should contain the latitude and longitude data for that one site. This also applies to the labels, markers, and colors. Any object in the index 0 slot will apply to that same point.)



In [97]:
latitude = [41.700937896518866, 41.73884644555692] 
longitude = [-87.99578103231573, -87.98744136114946]
labels = ['Tower', 'SCM']
markers = ['v', 'o']
colors = ['black', 'gray']

---

# Reading the data into PyART

To streamline the process of pulling and processing the radar files, we will create a function called read_radar_data. 

Within this function, some progress tracking code is implemented. Each time a file is successfully read, a message will be printed out letting you know what file in the order it is. This is useful to tell if your code is actually working.
An exception is added to this code so that the files marked MDM (shown on the list of filed compiled when pulling data) do not halt the process, and are instead skipped as they are not necessary.  


#### This function will be layed out in blocks, but will be shown in whole after each section is explained. 

In [101]:
def read_radar_data(file_path):
    try:
        with fs.open(file_path, 'rb') as f:
            radar_data = f.read()
        radar_file = BytesIO(radar_data)
        radar = pyart.io.read_nexrad_archive(radar_file)
        print(f"Successfully read radar data from {file_path}")
        return radar
    except Exception as e:
        print(f"Failed to read radar data from {file_path}: {e}")
        return None


#### The different radar products available to be plotted can be viewed below:

In [104]:
list(radar.fields)

['differential_reflectivity',
 'clutter_filter_power_removed',
 'spectrum_width',
 'velocity',
 'differential_phase',
 'cross_correlation_ratio',
 'reflectivity']

### Looping through the radar data

This code allows us to loop through each radar file read. A progress message is printed when a new file has started being processed. 
The if statement tells the code to skip files that are unable to be read.

In [107]:
for i, file in enumerate(files):
    print(f"Processing file {i+1}/{len(files)}: {file}")
    # Read radar data
    radar = read_radar_data(file)
    if radar is None:
        print(f"Skipping file {file} due to read error.")
        continue

Processing file 1/101: noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_040442_V06
Successfully read radar data from noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_040442_V06
Processing file 2/101: noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_040926_V06
Successfully read radar data from noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_040926_V06
Processing file 3/101: noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_041403_V06



KeyboardInterrupt



### Plotting code with added features and plotted points

The radar product being plotted can be changed based on the needs of the individual, but for this example, we will use reflectivity to make a singular gif. 

### Customizing the range of plotted data
Vmin and vmax represent the range of dBz values you'd like to plot on the radar. Sometimes, one may want to raise the lower limit to reduce clutter or nonmeteorological scatter that often appears as low reflectivity blobs near the radar. Basically the maximum and minimum values for your colorbar as well.

The sweep is the elevation being scanned. For example, sweep 0 is the lowest level scanned by the radar. 

### Counties

Counties can be added with the ax.add_feature line. Further additions can be made using cartopy.cfeature if needed. 

### Location plotting 
The for loop in this cell is used to plot the location data provided in the aforementioned variables. 

### Zooming 
The x and ylim functions will allow you to control the zoom on your plot. The grid on the plot is representative of latitude (y) and longitude (x) lines. For this example, we are zoomed in over the points.

In [109]:
fig = plt.figure(figsize=[12, 8])
    ax = plt.subplot(111, projection=ccrs.PlateCarree())
    display = pyart.graph.RadarMapDisplay(radar)
    try:
        display.plot_ppi_map('reflectivity',
                             sweep=0,
                             vmin=10,
                             vmax=65,
                             ax=ax,
                             title=f'Z for {os.path.basename(file)}',
                             cmap='pyart_ChaseSpectral')
    except Exception as e:
        print(f"Error plotting radar data for file {file}: {e}")
        plt.close(fig)
        continue

    # Set the extent to zoom in much closer and centered on the points
    plt.xlim(-88.1, -87.8)
    plt.ylim(41.6, 41.8)

    # Add counties
    ax.add_feature(USCOUNTIES, linewidth=0.5)
    
    
    for lat, lon, label, marker, color in zip(latitude, longitude, labels, markers, colors):
        ax.plot(lon, lat, marker, label=label, color=color, transform=ccrs.PlateCarree(), markersize=10)

    # Save the plot to a file
    filename = f'radar_frame_{i}.png'
    plt.tight_layout()
    plt.legend(loc='upper right', fontsize='large', title='Locations')
    plt.savefig(filename)
    plt.close(fig)

    # Add the file to the frames list
    frames.append(filename)
    print(f"Saved frame {i+1}/{len(files)}: {filename}")

IndentationError: unexpected indent (3221652724.py, line 2)

# Entire function for ease of access

In [114]:
def read_radar_data(file_path):
    try:
        with fs.open(file_path, 'rb') as f:
            radar_data = f.read()
        radar_file = BytesIO(radar_data)
        radar = pyart.io.read_nexrad_archive(radar_file)
        print(f"Successfully read radar data from {file_path}")
        return radar
    except Exception as e:
        print(f"Failed to read radar data from {file_path}: {e}")
        return None

# Loop through each radar file
for i, file in enumerate(files):
    print(f"Processing file {i+1}/{len(files)}: {file}")
    # Read radar data
    radar = read_radar_data(file)
    if radar is None:
        print(f"Skipping file {file} due to read error.")
        continue

    # Create a plot for the first sweep's reflectivity
    fig = plt.figure(figsize=[12, 8])
    ax = plt.subplot(111, projection=ccrs.PlateCarree())
    display = pyart.graph.RadarMapDisplay(radar)
    try:
        display.plot_ppi_map('reflectivity',
                             sweep=0,
                             vmin=10,
                             vmax=65,
                             ax=ax,
                             title=f'Z for {os.path.basename(file)}',
                             cmap='pyart_ChaseSpectral')
    except Exception as e:
        print(f"Error plotting radar data for file {file}: {e}")
        plt.close(fig)
        continue

    # Set the extent to zoom in much closer and centered on the points
    plt.xlim(-88.1, -87.8)
    plt.ylim(41.6, 41.8)

    # Add counties
    ax.add_feature(USCOUNTIES, linewidth=0.5)
    
    for lat, lon, label, marker, color in zip(latitude, longitude, labels, markers, colors):
        ax.plot(lon, lat, marker, label=label, color=color, transform=ccrs.PlateCarree(), markersize=10)

    # Save the plot to a file
    filename = f'radar_frame_{i}.png'
    plt.tight_layout()
    plt.legend(loc='upper right', fontsize='large', title='Locations')
    plt.savefig(filename)
    plt.close(fig)

    # Add the file to the frames list
    frames.append(filename)
    print(f"Saved frame {i+1}/{len(files)}: {filename}")

Processing file 1/101: noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_040442_V06
Successfully read radar data from noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_040442_V06
Saved frame 1/101: radar_frame_0.png
Processing file 2/101: noaa-nexrad-level2/2024/06/05/KLOT/KLOT20240605_040926_V06



KeyboardInterrupt



# GIF creation

We can now use the data to create a gif out of the frames we've appended to our list. 

This code also includes code to save the gif to your local directory.

Something to note, if you do not want to save the gif, you can get rid of this code.

As the frames are processed, they will temporarily save to your directory until the gif is made. They will save as PNG files, which are able to be opened and can be used to make sure everything is plotting on your figure correctly.



In [137]:
if frames:
    print("Creating animated GIF...")
    images = []
    for filename in frames:
        images.append(imageio.imread(filename))
    imageio.mimsave('radar_animation.gif', images, duration=10, loop=0)  # loop=0 for infinite loop

    # Clean up the saved frames
    for filename in frames:
        os.remove(filename)

    print("Animated GIF created as 'radar_animation.gif'")
else:
    print("No frames were generated.")


Creating animated GIF...
Animated GIF created as 'radar_animation.gif'


# Code in two parts

In [ ]:
import pyart
import fsspec
import matplotlib.pyplot as plt
import imageio
import os
from io import BytesIO
import warnings
import cartopy.crs as ccrs
from metpy.plots import USCOUNTIES
from scipy.interpolate import griddata
import numpy as np
warnings.filterwarnings("ignore")

fs = fsspec.filesystem("s3", anon=True)


date = "2024/06/05" #YYYY/MM/DD
station = "KLOT"

# Specify the start and end hours
start_hour = 4
end_hour = 10

# Generate the list of files for the specified range of hours
files = []
for hour in range(start_hour, end_hour + 1):
    hour_str = f"{hour:02d}"
    files += sorted(fs.glob(f"s3://noaa-nexrad-level2/{date}/{station}/{station}{date.replace('/', '')}_{hour_str}*"))
files

frames = []
latitude = [41.700937896518866, 41.73884644555692] 
longitude = [-87.99578103231573, -87.98744136114946]
labels = ['Tower', 'SCM']
markers = ['v', 'o']
colors = ['black', 'gray']

def read_radar_data(file_path):
    try:
        with fs.open(file_path, 'rb') as f:
            radar_data = f.read()
        radar_file = BytesIO(radar_data)
        radar = pyart.io.read_nexrad_archive(radar_file)
        print(f"Successfully read radar data from {file_path}")
        return radar
    except Exception as e:
        print(f"Failed to read radar data from {file_path}: {e}")
        return None

# Loop through each radar file
for i, file in enumerate(files):
    print(f"Processing file {i+1}/{len(files)}: {file}")
    # Read radar data
    radar = read_radar_data(file)
    if radar is None:
        print(f"Skipping file {file} due to read error.")
        continue

    # Create a plot for the first sweep's reflectivity
    fig = plt.figure(figsize=[12, 8])
    ax = plt.subplot(111, projection=ccrs.PlateCarree())
    display = pyart.graph.RadarMapDisplay(radar)
    try:
        display.plot_ppi_map('reflectivity',
                             sweep=0,
                             vmin=10,
                             vmax=65,
                             ax=ax,
                             title=f'Z for {os.path.basename(file)}',
                             cmap='pyart_ChaseSpectral')
    except Exception as e:
        print(f"Error plotting radar data for file {file}: {e}")
        plt.close(fig)
        continue

    # Set the extent to zoom in much closer and centered on the points
    plt.xlim(-88.1, -87.8)
    plt.ylim(41.6, 41.8)

    # Add counties
    ax.add_feature(USCOUNTIES, linewidth=0.5)
    
    for lat, lon, label, marker, color in zip(latitude, longitude, labels, markers, colors):
        ax.plot(lon, lat, marker, label=label, color=color, transform=ccrs.PlateCarree(), markersize=10)

    # Save the plot to a file
    filename = f'radar_frame_{i}.png'
    plt.tight_layout()
    plt.legend(loc='upper right', fontsize='large', title='Locations')
    plt.savefig(filename)
    plt.close(fig)

    # Add the file to the frames list
    frames.append(filename)
    print(f"Saved frame {i+1}/{len(files)}: {filename}")
if frames:
    print("Creating animated GIF...")
    images = []
    for filename in frames:
        images.append(imageio.imread(filename))
    imageio.mimsave('radar_animation.gif', images, duration=10, loop=0)  # loop=0 for infinite loop

    # Clean up the saved frames
    for filename in frames:
        os.remove(filename)

    print("Animated GIF created as 'radar_animation.gif'")
else:
    print("No frames were generated.")


# Summary

Within this example, we walked through how MetPy and PyART can be used to loop through NEXRAD level 2 data from a recent convective rainfall event and create an animated gif over a location.

## What comes next?

Following this will be some examples that allow us to create gifs with radar data visualized in less conventional forms, such as vertical columns. May be included on this page or a seperate one.